In [61]:
import pandas as pd

with open('C:/Users/win8/PycharmProjects/textmining/venv/1111stop.txt', encoding='utf8') as file:
    stopWord_list = [k.strip() for k in file.readlines()]

train_df = pd.read_csv('C:/Users/win8/PycharmProjects/textmining/venv/train.txt',sep='\t', header=None,)
train_df = train_df.astype(str)
train_df.shape

(2006, 2)

In [62]:
import jieba
import time

train_df.columns = ['分类', '文章']
stopword_list = [k.strip() for k in open('C:/Users/win8/Desktop/1111stop.txt', encoding='utf8').readlines() if k.strip() != '']
cutWords_list = []
i = 0
startTime = time.time()
for article in train_df['文章']:
    cutWords = [k for k in jieba.cut(article) if k not in stopword_list]
    i += 1
    if i % 1000 == 0:
        print('前%d篇文章分词共花费%.2f秒' %(i, time.time()-startTime))
    cutWords_list.append(cutWords)

前1000篇文章分词共花费53.45秒
前2000篇文章分词共花费89.40秒


In [63]:
with open('C:/Users/win8/PycharmProjects/textmining/venv/cutWords_list.txt', 'w',encoding='utf-8') as file: 
    for cutWords in cutWords_list:
        file.write(' '.join(cutWords) + '\n')
with open('C:/Users/win8/PycharmProjects/textmining/venv/cutWords_list.txt',encoding='utf-8') as file:
    cutWords_list = [k.split() for k in file.readlines()]

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(cutWords_list, min_df=20, max_df=0.3)

In [65]:
X = tfidf.fit_transform(train_df['文章'])
print(X.shape)

(2006, 6127)


In [66]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

train_df = pd.read_csv('C:/Users/win8/PycharmProjects/textmining/venv/train.txt', sep='\t', header=None)
train_df = train_df.astype(str)
labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(train_df[0].values)
y.shape

(2006,)

In [67]:
train_df[0].value_counts()
import time  
from sklearn import metrics  
import pickle as pickle  
import pandas as pd

In [69]:
#逻辑回归模型 LR
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)

0.9626865671641791

In [12]:
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
logistic_model.fit(train_X, train_y)
logistic_model.score(test_X, test_y)

In [70]:
#保存模型的代码块
import pickle

with open('C:/Users/win8/PycharmProjects/textmining/venv/lr_40_0.3.model', 'rb') as file:
    tfidf_model = pickle.load(file)
    tfidfVectorizer = tfidf_model['tfidfVectorizer']
    labelEncoder = tfidf_model['labelEncoder']
    logistic_model = tfidf_model['logistic_model']

In [71]:
import pandas as pd

train_df = pd.read_csv('C:/Users/win8/PycharmProjects/textmining/venv/train.txt', sep='\t', header=None)
train_df = train_df.astype(str)
X = tfidfVectorizer.transform(train_df[1])
y = labelEncoder.transform(train_df[0])

In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
cv_split = ShuffleSplit(n_splits=5, test_size=0.3)
score_ndarray = cross_val_score(logistic_model, X, y, cv=cv_split)
print(score_ndarray)
print(score_ndarray.mean())

[0.96179402 0.95016611 0.94352159 0.9551495  0.94684385]
0.9514950166112957


In [73]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import pandas as pd

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
logistic_model.fit(train_X, train_y)
predict_y = logistic_model.predict(test_X)
pd.DataFrame(confusion_matrix(test_y, predict_y), 
             columns=labelEncoder.classes_, 
             index=labelEncoder.classes_)

,互联网,体育,军事,娱乐,电商,财经,资讯,零售
互联网,51,0,0,0,0,0,0,0
体育,0,45,0,0,0,0,0,0
军事,0,0,43,0,0,0,0,0
娱乐,0,0,0,51,0,0,0,0
电商,1,0,0,1,38,0,1,0
财经,0,0,1,1,2,51,1,0
资讯,0,0,3,0,0,0,54,0
零售,0,0,0,0,1,2,1,54


In [74]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(classification_report(test_y, predict_y))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        51
           1       1.00      1.00      1.00        45
           2       0.91      1.00      0.96        43
           3       0.96      1.00      0.98        51
           4       0.93      0.93      0.93        41
           5       0.96      0.91      0.94        56
           6       0.95      0.95      0.95        57
           7       1.00      0.93      0.96        58

   micro avg       0.96      0.96      0.96       402
   macro avg       0.96      0.96      0.96       402
weighted avg       0.96      0.96      0.96       402



In [75]:
import pandas as pd

test_df = pd.read_csv('C:/Users/win8/PycharmProjects/textmining/venv/train.txt', sep='\t', header=None)
test_X = tfidfVectorizer.transform(test_df[1])
test_y = labelEncoder.transform(test_df[0])
predict_y = logistic_model.predict(test_X)
print(classification_report(test_y, predict_y))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       252
           1       1.00      1.00      1.00       250
           2       0.97      1.00      0.98       250
           3       0.99      1.00      1.00       252
           4       0.98      0.98      0.98       250
           5       0.99      0.96      0.97       251
           6       0.98      0.98      0.98       250
           7       0.99      0.98      0.99       251

   micro avg       0.99      0.99      0.99      2006
   macro avg       0.99      0.99      0.99      2006
weighted avg       0.99      0.99      0.99      2006

